In [1]:
from bs4 import BeautifulSoup
import requests

In [29]:
#url = 'http://archiveofourown.org/works/'+str(22336816)+'/kudos'  
#url = 'http://archiveofourown.org/works/'+str(14737352)+'/kudos' 
url = 'http://archiveofourown.org/works/'+str(22336817)+'/kudos' # 404
headers = {'user-agent' : 'Homemade Bot; rebecca.sanjabi@gmail.com'}
    
req = requests.get(url, headers=headers)
src = req.text
soup = BeautifulSoup(src, 'html.parser')
print(url)

http://archiveofourown.org/works/22336817/kudos


In [30]:
def get_kudos(soup):
    try:
        my_list = soup.find(id="kudos").find_all('a')
        return [x.text for x in set(my_list)]
    except AttributeError as e:
        return []

In [32]:
if req.status_code != 200:
    print('Access Denied')
my_kudo_list = get_kudos(soup)
unique_list = sorted(my_kudo_list)
for el in unique_list:
    print(el)
print(len(unique_list))

Access Denied
0
